# Problem Statement

## Retinal Fundus Multi-Disease Image Dataset

# Solution Statement

## 1. Importing Required Modules

In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm

## 2. Reading the CSV train, Test, and Evaluation data

In [3]:
train =pd.read_csv('../Workspace/RFMDI Dataset/Training_Set/RFMiD_Training_Labels.csv')
test  =pd.read_csv('../Workspace/RFMDI Dataset/Test_Set/RFMiD_Testing_Labels.csv')
val  =pd.read_csv('../Workspace/RFMDI Dataset/Evaluation_Set/RFMiD_Validation_Labels.csv')

In [8]:
train_abnrml_path='../Workspace/RFMDI Dataset/Training_Set/Training/abnrml'
train_nrml_path='../Workspace/RFMDI Dataset/Training_Set/Training/nrml'
#for i in range(len(train['ID'])):
#    if train['Disease_Risk'][i] == 1:
#        shutil.copy(os.path.join(seg_path,str(train['ID'][i])+'.png'),train_abnrml_path)
#    elif train['Disease_Risk'][i] == 0:
#        shutil.copy(os.path.join(seg_path,str(train['ID'][i])+'.png'),train_nrml_path)

x_train=[]
for img in tqdm(os.listdir(train_abnrml_path)):    
    image_path=train_abnrml_path+"/"+img
    img_arr=cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(224,224))
    x_train.append(img_arr)

for img in tqdm(os.listdir(train_nrml_path)):    
    image_path=train_nrml_path+"/"+img
    img_arr=cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(224,224))
    x_train.append(img_arr)

100%|████████████████████████████████████████████████████████████████████████████████| 401/401 [00:39<00:00, 10.23it/s]


In [10]:
test_abnrml_path='../Workspace/RFMDI Dataset/Test_Set/Test/abnrml'
test_nrml_path='../Workspace/RFMDI Dataset/Test_Set/Test/nrml'
seg_path='../Workspace/RFMDI Dataset/Test_Set/Test'
for i in range(len(test['ID'])):
    if test['Disease_Risk'][i] == 1:
        shutil.copy(os.path.join(seg_path,str(test['ID'][i])+'.png'),test_abnrml_path)
    elif test['Disease_Risk'][i] == 0:
        shutil.copy(os.path.join(seg_path,str(test['ID'][i])+'.png'),test_nrml_path)

x_test=[]
for img in tqdm(os.listdir(test_abnrml_path)):    
    image_path=test_abnrml_path+"/"+img
    img_arr=cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(224,224))
    x_test.append(img_arr)

for img in tqdm(os.listdir(test_nrml_path)):    
    image_path=test_nrml_path+"/"+img
    img_arr=cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(224,224))
    x_test.append(img_arr)

100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:17<00:00,  7.45it/s]


In [11]:
val_abnrml_path='../Workspace/RFMDI Dataset/Evaluation_Set/Validation/abnrml'
val_nrml_path='../Workspace/RFMDI Dataset/Evaluation_Set/Validation/nrml'
seg_path='../Workspace/RFMDI Dataset/Test_Set/Test'
for i in range(len(test['ID'])):
    if val['Disease_Risk'][i] == 1:
        shutil.copy(os.path.join(seg_path,str(val['ID'][i])+'.png'),val_abnrml_path)
    elif val['Disease_Risk'][i] == 0:
        shutil.copy(os.path.join(seg_path,str(val['ID'][i])+'.png'),val_nrml_path)

x_val=[]
for img in tqdm(os.listdir(val_abnrml_path)):    
    image_path=val_abnrml_path+"/"+img
    img_arr=cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(224,224))
    x_val.append(img_arr)

for img in tqdm(os.listdir(val_nrml_path)):    
    image_path=val_nrml_path+"/"+img
    img_arr=cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(224,224))
    x_val.append(img_arr)

100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:18<00:00,  7.40it/s]


In [12]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [13]:
train_x.shape
test_x.shape
val_x.shape

(640, 224, 224, 3)

In [14]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_set = train_datagen.flow_from_directory('../Workspace/RFMDI Dataset/Training_Set/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
testing_set = train_datagen.flow_from_directory('../Workspace/RFMDI Dataset/Test_Set/Test',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
valuation_set = train_datagen.flow_from_directory('../Workspace/RFMDI Dataset/Evaluation_Set/Validation',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

Found 1920 images belonging to 2 classes.
Found 640 images belonging to 2 classes.
Found 640 images belonging to 2 classes.


In [19]:
training_set.class_indices
testing_set.class_indices
valuation_set.class_indices

{'abnrml': 0, 'nrml': 1}

In [20]:
train_y=training_set.classes
test_y=testing_set.classes
val_y=valuation_set.classes

In [21]:
train_y.shape
test_y.shape
val_y.shape

(640,)

In [22]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [23]:
# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [24]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [25]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

prediction = Dense(2, activation='softmax')(x)

In [26]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [27]:
# tell the model what cost and optimization method to use
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

#Early stopping to avoid overfitting of model

In [31]:
# fit the model
history = tqdm(model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True))

Epoch 1/10
60/60 [==============================] - 1151s 19s/step - loss: 0.2726 - accuracy: 0.8786 - val_loss: 0.8229 - val_accuracy: 0.6703
Epoch 2/10
60/60 [==============================] - 1122s 19s/step - loss: 0.3002 - accuracy: 0.8568 - val_loss: 0.7387 - val_accuracy: 0.7734
Epoch 3/10
60/60 [==============================] - 1124s 19s/step - loss: 0.2793 - accuracy: 0.8760 - val_loss: 0.7823 - val_accuracy: 0.7469
Epoch 4/10
60/60 [==============================] - 1123s 19s/step - loss: 0.2423 - accuracy: 0.8979 - val_loss: 0.7430 - val_accuracy: 0.7812
Epoch 5/10
60/60 [==============================] - 1114s 19s/step - loss: 0.2406 - accuracy: 0.8984 - val_loss: 0.7774 - val_accuracy: 0.7719
Epoch 6/10
60/60 [==============================] - 1114s 19s/step - loss: 0.2831 - accuracy: 0.8802 - val_loss: 0.8149 - val_accuracy: 0.7578
Epoch 7/10
60/60 [==============================] - 1113s 19s/step - loss: 0.2492 - accuracy: 0.8969 - val_loss: 0.7834 - val_accuracy: 0.7656

0it [00:00, ?it/s]